In [ ]:
from geo_dist_prep.data.populate_tree import populate_tree

globe = populate_tree('..')

In [ ]:
from geo_dist_prep.tree.globe import Globe
from geo_dist_prep.tree.pos import Pos

southport = 26700924
lat1, lon1 = 53.647599, -3.004175  # southport

newglobe = Globe()
address = list(newglobe.get_address(lat1, lon1))

print(address)

southport = globe.find(address)
str(southport)

In [ ]:

def plot_node(node, depth: int = 6, colour="black"):
    """
    Draw this triangle and its children to the given depth.
    """
    import matplotlib.pyplot as plt
    from geo_dist_prep.tree.node import Node

    node: Node = node

    verts = node.vertices
    x = [vert[0] for vert in verts] + [verts[0][0]]
    y = [vert[1] for vert in verts] + [verts[0][1]]

    plt.plot(x, y, color=colour)
    
    # if node.data:
    #     plt.text(
    #         node.lat,
    #         node.lon,
    #         str(node),
    #         color=colour,
    #     )

    if depth > 1:
        for idx in [Pos.LEFT_POINT, Pos.RIGHT_POINT, Pos.TIP, Pos.CENTER]:
            child = node.relations.get(idx)
            if child:
                plot_node(child, depth - 1, colour)


globe.plot()
region = globe.relations[6]
plot_node(globe.relations[6], colour='green')
plot_node(globe.relations[7], colour='green')
plot_node(globe.relations[11], colour='green')

#region.parent.plot()
#region.plot(depth=3, colour='green')
#region.relations[Pos.RIGHT_EDGE].plot(colour='red', depth=3)
#region.relations[Pos.BASE].plot(colour='red', depth=3)
#print('siblings:', region.parent.children)
#print('neighbours:', region.neighbours)
#print('parent neighbours:', region.parent.neighbours)
#southport.parent.parent.neighbours[1].plot(colour='red')
#southport.parent.width * 113.5

#address = (i for i in ('0/3/3/2/0/0/2/1' '/-3/-3/-1/0/2').split('/'))
#address = (int(i) for i in ('0/3/3/2/0/3/0/2/-0').split('/')) 
#right_region = globe.find(address)
#right_region.plot(depth=3, colour='red')

print("region:", repr(region))
#print("right region neightbours:", right_region.neighbours)


In [ ]:
from geo_dist_prep.tree.node import Node
from geo_dist_prep.tree.globe import Globe
from geo_dist_prep.tree.pos import Pos


s = southport.relations['c']
s.plot(depth=1)

s.relations


In [ ]:
from geo_dist_prep.tree.node import Node

def node_stats(node: Node, stats: dict):
    stats['total'] += 1
    if node.depth not in stats['by depth']:
        stats['by depth'][node.depth] = 0
    stats['by depth'][node.depth] += 1

    neighbours = sum((
        1 if node.relations.get(Pos.LEFT_EDGE) else 0,
        1 if node.relations.get(Pos.RIGHT_EDGE) else 0,
        1 if node.relations.get(Pos.BASE) else 0
    ))

    if neighbours:
        stats['total neighbours'] += neighbours
    else:
        stats['without neighbours'] += 1

    if not node.data:
        stats['structural'] += 1
    
    children = (c for c in (
        node.relations.get(Pos.LEFT_POINT),
        node.relations.get(Pos.RIGHT_POINT),
        node.relations.get(Pos.TIP)
    ) if c)

    for child in children:
        node_stats(child, stats)


def tree_stats(globe: Globe):
    stats = {
        'total': 0,
        'by depth': {

        },
        'without neighbours': 0,
        'total neighbours': 0,
        'structural': 0
    }
    for name in globe.relations:
        if name not in list(Pos):
            node_stats(globe.relations[name], stats)
    return stats

stats = tree_stats(globe)

display(stats)